# irodsConnector 

## Create an iRODS session (connection to iRODS server)

In [ ]:
from irodsConnector.session import Session
import os, json

### Password authentication

In [ ]:
with open(os.path.expanduser("~/.irods/irods_environment.json"), "r") as f:
    ienv = json.load(f)
password = "<YOUR PASSWORD>"
session = Session(irods_env=ienv, password=password)

### Using the cached password ~/.irods/.rodsA

In [ ]:
session = Session(irods_env_path=os.path.expanduser("~/.irods/irods_environment.json"))

### Checking some session parameters

In [ ]:
print(session.username)
print(session.default_resc) # the resource to which data will be uploaded
print(session.zone) # default home for iRODS /zone/home/username
print(session.server_version)

## Resources and handling resources

In [ ]:
from irodsConnector.resources import Resources
resources = Resources(session)

### Check if default resource exists

In [ ]:
default_resc = resources.get_resource(session._irods_env.get("irods_default_resource", ""))
print(default_resc.name)
print(default_resc.free_space) # Metadata how much bytes are left on resource, None if not set
print(default_resc.type) # Storage policy

### Listing resources

In [ ]:
print(resources.resources()) # all resources
print(resources.root_resources) # all writeable resources (name, status, free space, context)

### Retrieve current free space
In contrast to `resc.free_space` the function `get_free_space` accumulates all free space in the subtree starting with the resource as parent.

In [ ]:
resources.get_free_space(session._irods_env.get("irods_default_resource", "")) # default resource name